In [1]:
####

# Code generated for "Bill of Lading" dataset registered under "Bill of Lading (PIERS)" package

# Documentation and Installations Instructions link: https://catalogue.datalake.ihsmarkit.com (please follow the links to "Documentation")

# This code is compatible with latest version of the Data Lake command line interface hosted on pypi.org: https://pypi.org/project/dli/

# To run with python interpreter (preferably using 3.x version)

####



# Import Libraries

import dli
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib
from sklearn.linear_model import LinearRegression
import os, glob
from os import listdir
import gc


print('#All Libaries are imported already#')

#All Libaries are imported already#


In [2]:
# Function for transforming cds
def exp(x):
        return 2.71828182845904523536**x
def transform_cds(df,ticker,year,month):
    # Filter according to the Tenor column and extract only the rows of Tenor under 1 years
    df = df[df.Tenor.isin(['3M','6M','9M','1Y'])]
    # Calculate the Probability of Default
    mapping = {'3M':0.25,'6M':0.5,'9M':0.75,'1Y':1}
    df = df.replace({'Tenor': mapping})
    df['PD']=1-exp(df['Tenor']*df['ConvSpread']/(df['RealRecovery']-1))
    df = df[df['Ticker'].isin(ticker)]
    df['Year'] = year
    df['Month'] = month
    df = df[(df['PrimaryCoupon']== True) & (df['PrimaryCurve']== True)]
    return df

# Extract file name of the CDS datasets
def cds_getfilepath(i,j):
    a = "%02d" % (j,)
    k=1
    dirName = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\CDSSingleNamePricingSensitivitiesandLiquidity\as_of_date={}-{}-0{}".format(i,a,k)
    while ((os.path.exists(dirName) == False) and (k != 10)):
        k = k+1
        dirName = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\CDSSingleNamePricingSensitivitiesandLiquidity\as_of_date={}-{}-0{}".format(i,a,k)
    CDS_Files = [file for file in listdir(dirName) if file.endswith('.snappy.parquet')]
    CDS_File = CDS_Files[0]
    path = os.path.join(dirName, CDS_File)
    return path

In [3]:
#Filter function for export
BoL_ticker = 'matched_fcompany_ihsm_ticker'
tickers = pd.read_excel(rf'C:\Users\Thomas TH Chow\Desktop\Datalake\Tickers\IHS {BoL_ticker} Common List.xlsx')['Common Ticker'].to_list()

# main function to transforming all year cds data of exporting company
for i in range(2001,2021):
    for j in range(1,13):
        try:
            cds_df = pd.read_parquet(cds_getfilepath(i,j))
            cds_df = transform_cds(cds_df,tickers,i,j)
            # Store the completed datasets to local drive
            j = "%02d" % (j,)
            output_path = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\CDS\cds_df as_of_date={}-{}.parquet.gzip".format(i,j)
            cds_df.to_parquet(output_path,
                          compression='gzip')
            del cds_df
            gc.collect()
        except:
            print('All available files have been transformed.')
            continue

All available files have been transformed.


In [6]:
#Filter function for import
BoL_ticker = 'matched_ntfcompany_ihsm_ticker'
tickers = pd.read_excel(rf'C:\Users\Thomas TH Chow\Desktop\Datalake\Tickers\IHS {BoL_ticker} Common List.xlsx')['Common Ticker'].to_list()

# main function to transforming all year cds data of importing company
for i in range(2001,2021):
    for j in range(1,13):
        try:
            cds_df = pd.read_parquet(cds_getfilepath(i,j))
            cds_df = transform_cds(cds_df,tickers,i,j)
            # Store the completed datasets to local drive
            j = "%02d" % (j,)
            output_path = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Import Data\CDS\cds_df as_of_date={}-{}.parquet.gzip".format(i,j)
            cds_df.to_parquet(output_path,
                          compression='gzip')
            del cds_df
            gc.collect()
        except:
            print('All available files have been transformed.')
            continue

All available files have been transformed.


Below are the codes for generating data of a specific company -- optional for internal discussion 

In [ ]:
#Sort and Filter all companies with potentials of default and output two company level Excel file for exploration
Tickers = default_df['Ticker'].unique().tolist()
output_path = r"C:\Users\Thomas TH Chow\Desktop\DataLake Team Email 12112020\Company Going to default"

for Ticker in Tickers:
    input_path = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\CDS datasets"
    cds_files = [file for file in listdir(input_path) if file.endswith('.parquet.gzip')]
    for i in range (0,len(cds_files)):
        cds_file = cds_files[i]
        path = os.path.join(input_path, cds_file)
        print(path)
        temp_df = pd.read_parquet(path)
        print(temp_df.shape)
        temp_df = temp_df[temp_df['Ticker'] == Ticker]
        temp_df.shape
        if i == 0:
            default_df = temp_df
        else:
            default_df = default_df.append(temp_df, sort=False)
        default_df.shape
        del temp_df
        gc.collect()
    default_df.shape
    default_df.to_excel(f'{output_path}\\{Ticker}.xlsx')

    input_path = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Bill of Lading"
    Bill_Temps = [file for file in listdir(input_path) if file.endswith('.parquet.gzip')]
    for j in range (0,len(Bill_Temps)):
        Bill_Temp = Bill_Temps[j]
        path = os.path.join(input_path, Bill_Temp)
        print(path)
        Temp_df = pd.read_parquet(path)
        print(Temp_df.shape)
        Temp_df = Temp_df[Temp_df['matched_ntfcompany_ihsm_ticker'] == Ticker]
        if (j==0):
            default_bill_df = Temp_df
        else: 
            default_bill_df = default_bill_df.append(Temp_df, sort=False)
        del Temp_df
        gc.collect()
    default_bill_df.shape
    default_bill_df.to_excel(f'{output_path}\\{Ticker}_bill.xlsx')